In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, size_average=False).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [3]:
# Training settings
parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                    help='input batch size for training (default: 64)')
parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                    help='input batch size for testing (default: 1000)')
parser.add_argument('--epochs', type=int, default=10, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                    help='learning rate (default: 0.01)')
parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                    help='SGD momentum (default: 0.5)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')
args = parser.parse_args('')
use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)


model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

for epoch in range(1, args.epochs + 1):
    train(args, model, device, train_loader, optimizer, epoch)
    test(args, model, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.373651
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.310517
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.281828
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.315809
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.235439
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.234249
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.226109
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.228646
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.132810
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.113179
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.030113
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.877119
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.894014
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.725610
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.739437
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.533461
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.549235
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.498123
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.502510
Train Epoch: 1 [12160/60000 (20%)]	Loss: 1.326280
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.364253
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.300039
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.426649
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.365356
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.440504
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.222699
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.200448
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.284925
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.332481
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.319938
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.436406
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.486359
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.234702
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.311248
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.288041
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.239962
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.204596
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.276910
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.244168
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.397649


Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.154859
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.372721
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.143620
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.246007
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.153609
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.297043
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.264638
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.281437
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.267597
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.271944
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.084459
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.429408
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.133438
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.267528
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.184385
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.196758
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.301186
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.320578
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.220839
Train Epoch: 4 [41600/60000 (69%)]	Loss: 0.333038


Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.204757
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.189321
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.335364
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.139522
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.114080
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.188998
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.238996
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.172923
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.190651
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.257767
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.371477
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.408340
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.226279
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.181559
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.133435
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.165798
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.202606
Train Epoch: 6 [24960/60000 (42%)]	Loss: 0.127531
Train Epoch: 6 [25600/60000 (43%)]	Loss: 0.267071
Train Epoch: 6 [26240/60000 (44%)]	Loss: 0.282775



Test set: Average loss: 0.0665, Accuracy: 9798/10000 (98%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.290872
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.115379
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.129628
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.174925
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.113587
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.297224
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.121353
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.157989
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.064785
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.189490
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.272315
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.062669
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.377775
Train Epoch: 8 [8320/60000 (14%)]	Loss: 0.074211
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.139135
Train Epoch: 8 [9600/60000 (16%)]	Loss: 0.295423
Train Epoch: 8 [10240/60000 (17%)]	Loss: 0.104255
Train Epoch: 8 [10880/60000 (18%)]	Loss: 0.142420
Train Epoch: 8 [11520/60000 (19%)]	Loss: 0.085257
Train Epoch: 8 [12

Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.153239
Train Epoch: 9 [45440/60000 (76%)]	Loss: 0.178236
Train Epoch: 9 [46080/60000 (77%)]	Loss: 0.143179
Train Epoch: 9 [46720/60000 (78%)]	Loss: 0.204229
Train Epoch: 9 [47360/60000 (79%)]	Loss: 0.106992
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.127463
Train Epoch: 9 [48640/60000 (81%)]	Loss: 0.161283
Train Epoch: 9 [49280/60000 (82%)]	Loss: 0.119853
Train Epoch: 9 [49920/60000 (83%)]	Loss: 0.161039
Train Epoch: 9 [50560/60000 (84%)]	Loss: 0.157527
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.059951
Train Epoch: 9 [51840/60000 (86%)]	Loss: 0.111331
Train Epoch: 9 [52480/60000 (87%)]	Loss: 0.127275
Train Epoch: 9 [53120/60000 (88%)]	Loss: 0.228970
Train Epoch: 9 [53760/60000 (90%)]	Loss: 0.193132
Train Epoch: 9 [54400/60000 (91%)]	Loss: 0.127023
Train Epoch: 9 [55040/60000 (92%)]	Loss: 0.144248
Train Epoch: 9 [55680/60000 (93%)]	Loss: 0.056340
Train Epoch: 9 [56320/60000 (94%)]	Loss: 0.209366
Train Epoch: 9 [56960/60000 (95%)]	Loss: 0.179162


Process Process-18:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.5/dist-packages/torch/utils/data/dataloader.py", line 57, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/usr/local/lib/python3.5/dist-packages/torch/utils/data/dataloader.py", line 57, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/usr/local/lib/python3.5/dist-packages/torchvision/datasets/mnist.py", line 79, in __getitem__
    if self.target_transform is not None:
KeyboardInterrupt
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-beff051cd4e7>", line 47, in <module>
    test(args, model, device, test_loader)
  File "<ipython-input-1-42844a641b86>", line 46, in test
    for data, target in test_loader:
  File "/usr/local/lib/python3.5/dist-packages/torch/utils/data/dataloader.py", line 280, in __next__
    idx, batch = self._get_batch()
  File "/usr/local/lib/python3.5/dist-packages/torch/utils/data/dataloader.py", line 259, in _get_batch
    return self.data_queue.get()
  File "/usr/lib/python3.5/queue.py", line 164, in get
    self.not_empty.wait()
  File "/usr/lib/python3.5/threading.py", line 293, in wait
    waiter.acquire()
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/

KeyboardInterrupt: 